In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import Polynomial as P
from numpy.polynomial import Chebyshev as C
from numpy.polynomial import Legendre as L

In [2]:
inte_p = lambda p,a,b: p.integ()(b) - p.integ()(a)
def Gaussian_1D_hankel(n):
    # moments = np.array([inte_p(C.basis(r) , -1 , 1) for r in range(2 * n)])

    moments = np.array([(1**(r+1) - (-1)**(r+1)) / (r+1) for r in range(2 * n)])

    # moments = np.array([inte_p(L.basis(r) , -1 , 1) for r in range(2 * n)])

    M = np.array([moments[i:i+n] for i in range(n)])
    b = moments[n:]
    coef = np.linalg.solve(M, b)
    p = P(np.insert(-coef , n , 1))
    nodes = p.roots()
    V = np.array([nodes**i for i in range(n)])
    weights = np.linalg.solve(V , moments[:n])
    return nodes,weights

In [3]:
n = 6
real_n , real_w = np.polynomial.legendre.leggauss(n)
n , w = Gaussian_1D_hankel(n)
real_n , n , real_w , w


(array([-0.93246951, -0.66120939, -0.23861919,  0.23861919,  0.66120939,
         0.93246951]),
 array([-0.93246951, -0.66120939, -0.23861919,  0.23861919,  0.66120939,
         0.93246951]),
 array([0.17132449, 0.36076157, 0.46791393, 0.46791393, 0.36076157,
        0.17132449]),
 array([0.17132449, 0.36076157, 0.46791393, 0.46791393, 0.36076157,
        0.17132449]))

In [4]:
n = 13
real_n , real_w = np.polynomial.legendre.leggauss(n)
n , w = Gaussian_1D_hankel(n)
np.linalg.norm(n.real),np.linalg.norm(real_n)

(2.4979991992969732, 2.4979991993593593)

In [5]:
def is_psd_for_sym(A):
    m , n = A.shape
    if m == 1 or n == 1:
        return True,'0'
    # Determine if it's symmetry

    # Case 1
    if A[0,0] < 0:
        return False,'1'
    
    # Case 2
    elif A[0,0] == 0:
        for j in range(n):
            if A[0,j] != 0:
                return False,'2'
        return is_psd_for_sym(A[1: , 1:])[0],'2'
    
    # Case 3
    elif A[0,0] > 0:
        B = A.copy()
        for i in range(1,m):
            B[i] -= A[i,0] / A[0,0] * A[1]
        return is_psd_for_sym(B[1:,1:])[0],'3'
    
def is_psd_eig(M, atol=1e-12):
    """
    Returns True if all eigenvalues of M are >= -atol.
    atol is a numerical tolerance to allow for floating-point inaccuracies.
    """
    w, _ = np.linalg.eigh(M)  # Eigenvalues in ascending order
    return np.all(w >= -atol)    
    
